# *Report*

<table>
    <tr>
        <td> Topic: A Star Search Algorithm</td>
    </tr>
    <tr>
        <td>Submitted By: </td>
        <td>Md. Mubtasim Fuad</td>
    </tr>
    <tr>
        <td>Submitted To: </td>
        <td>Dr. Nasima Begum</td>
    </tr>
</table>

# *Introduction:*
Popular path-finding algorithms like the A* search algorithm are used often in computer science and artificial intelligence. It is a kind of informed search algorithm that combines the advantages of Greedy Best-First-Search with Dijkstra's algorithm. I'll utilize the A* search method to determine the shortest route from my house to UAP for this assignment. I'll utilize a graph dataset containing place names, latitudes, and longitudes connected by edges, where each edge denotes a route from site A to location B.


##### Importing Necessary Packages

In [1]:
!pip install pandas
!pip install haversine
!pip install folium
!pip install networkx
!pip install pyvis

In [2]:
import math

def haversine(coord1, coord2):
    earth_radius = 6371  # kilometers
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + \
        math.cos(phi1) * math.cos(phi2) * \
        math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = earth_radius * c  
    return distance

In [3]:
import pandas as pd
import folium
from folium.features import DivIcon

##### Importing our location data which contains location Location_Name,Latitude and Longitude.

In [4]:
location_data=pd.read_csv('./location_area.csv')

##### Location Dataset

In [5]:
location_data

,Location,Latitude,Longitude
0,Organ Lily,23.798547,90.370238
1,Shewrapara,23.789499,90.376024
2,Agargaon,23.778539,90.380004
3,Chandrima Uddan,23.765273,90.383162
4,Khamar Bari,23.758967,90.383925
5,Bijoy Sarani,23.764370,90.388905
6,Farmgate,23.757688,90.390176
7,Kamal Sarani,23.786766,90.366914
8,National Museum,23.779784,90.368032
9,Sher-e-Bangla Road,23.775074,90.374060


In [6]:
source=location_data.iloc[0]
goal=location_data.iloc[-1]
print(source)
print("\n")
print(goal)

Location     Organ Lily
Latitude      23.798547
Longitude     90.370238
Name: 0, dtype: object


Location     University of Asia Pacific
Latitude                      23.754956
Longitude                     90.389039
Name: 10, dtype: object


## MAP
##### Lets plot our data using folium library and marker green indicates starting position and red indicates destination

In [7]:
location_data_excluded=location_data.iloc[1:location_data.shape[0]-1]

In [11]:
m = folium.Map(location=[23.745268183480682,90.36089271175031], zoom_start=12, tiles="OpenStreetMap")

points=[]

folium.Marker(
            [source[1],source[2]], popup="🏘️", tooltip=source[0],
            icon=folium.Icon(color='green')).add_to(m)

folium.Marker(
            [goal[1],goal[2]], popup="🏢", tooltip=goal[0],
            icon=folium.Icon(color='red')).add_to(m)



for i in range(0,location_data_excluded.shape[0]):
        folium.Marker(
            [location_data_excluded.iloc[i][1],location_data_excluded.iloc[i][2]], popup=location_data_excluded.iloc[i][0], tooltip=location_data_excluded.iloc[i][0]
        ).add_to(m)
          
m



##### Importing our location graph dataset

In [12]:
location_graph=pd.read_csv('./location_graph.csv')

In [13]:
graph=dict()
path_costs=[]
for i in range(0,location_graph.shape[0]):
    loc_1=location_data[location_data['Location']==location_graph.iloc[i][0]].values.tolist()
    loc_2=location_data[location_data['Location']==location_graph.iloc[i][1]].values.tolist()
    if len(loc_1)>0 and len(loc_2)>0:
        loc_1_name=loc_1[0][0]
        loc_2_name=loc_2[0][0]
        loc_1=loc_1[0][1:]
        loc_2=loc_2[0][1:]
        path_cost=haversine(loc_1, loc_2)
        if loc_1_name in graph:
            graph[loc_1_name].append((loc_2_name,path_cost))
            path_costs.append(path_cost)
        else:
            graph[loc_1_name]=[(loc_2_name,path_cost)]
            path_costs.append(path_cost)

In [14]:
location_graph["Path Cost(KM)"]=path_costs
location_graph.to_csv('./location_map.csv')
location_graph

,Location_A,Location_B,Path Cost(KM)
0,Organ Lily,Shewrapara,1.165738
1,Organ Lily,Kamal Sarani,1.352908
2,Shewrapara,Agargaon,1.284150
3,Agargaon,Chandrima Uddan,1.509793
4,Chandrima Uddan,Khamar Bari,0.705409
5,Chandrima Uddan,Bijoy Sarani,0.593052
6,Khamar Bari,Farmgate,0.651892
7,Bijoy Sarani,Farmgate,0.754207
8,Farmgate,University of Asia Pacific,0.325022
9,Kamal Sarani,National Museum,0.784726


# *Heuristic Function:*
In A* search, a heuristic function is used to estimate the distance or cost from the current node to the goal node. This heuristic function is denoted by h(n) and is defined as:

h(n) = estimated cost from node n to the goal node

***Equirectangular Approximation as Heuristic function:***
The equirectangular approximation is a simple method for approximating the distance between two points on the surface of a sphere, such as the Earth. It is based on the assumption that the surface of the sphere can be projected onto a flat, rectangular plane in a way that preserves the relative distances between points.

\begin{align*}
x &= \Delta\lambda \cdot \cos \phi_m \
y &= \Delta\phi \
d &= R \cdot \sqrt{x^2 + y^2}
\end{align*}

where:
- $x$ is the east-west distance between two points on the surface of a sphere, in units of length (e.g., meters, kilometers, miles).
- $y$ is the north-south distance between two points on the surface of a sphere, in units of length.
- $d$ is the great-circle distance between two points on the surface of a sphere, in units of length.
- $\Delta\lambda$ is the difference in longitude between two points, in radians.
- $\phi_m$ is the average latitude between two points, in radians.
- $\Delta\phi$ is the difference in latitude between two points, in radians.
- $R$ is the radius of the sphere, in units of length.

In this projection, the latitude and longitude coordinates of each point are mapped to their respective x and y coordinates on the plane, with the x-axis representing the longitude and the y-axis representing the latitude. The distance between two points on the plane can then be calculated using the Pythagorean theorem:

To use this formula as a heuristic function in A* search, we would need to calculate the distance between the current node and the goal node using the above formula, where the current node's latitude and longitude would be used for lat1 and lon1, and the goal node's latitude and longitude would be used for lat2 and lon2. This distance value would then be used as the estimated cost from the current node to the goal node.

While the equirectangular approximation is computationally simple and fast, it does have limitations. One major limitation is that it becomes less accurate as the distance between the two points increases, particularly at higher latitudes. This is because the projection stretches the distances in the east-west direction as the latitude increases, leading to distortions in the distances*

In [15]:
import math

def equirectangular_approximation(loc_1, loc_2):
    R = 6371 # radius of the Earth in kilometers
    lat1=loc_1[0]
    lat2=loc_2[0]
    lon1 = loc_1[1]
    lon2 = loc_2[1] 

    phi1 = lat1 * math.pi / 180
    phi2 = lat2 * math.pi / 180
    delta_phi = (lat2 - lat1) * math.pi / 180
    delta_lambda = (lon2 - lon1) * math.pi / 180
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    
    return d


#### *Adding the above heurisitic function to our dataset*


In [16]:
heuristics =dict()
goal_loc=(float(goal[1]),float(goal[2]))
hs=[]
for i in range(0,location_data.shape[0]):
    hs.append(equirectangular_approximation(goal_loc,(float(location_data.iloc[i][1]),float(location_data.iloc[i][2]))))
    heuristics[location_data.iloc[i][0]]=hs[-1]

#### *Dataset with heurisitics values*

In [17]:
location_data["heurisitics"]=hs
location_data

,Location,Latitude,Longitude,heurisitics
0,Organ Lily,23.798547,90.370238,5.211022
1,Shewrapara,23.789499,90.376024,4.062888
2,Agargaon,23.778539,90.380004,2.778857
3,Chandrima Uddan,23.765273,90.383162,1.293719
4,Khamar Bari,23.758967,90.383925,0.685438
5,Bijoy Sarani,23.764370,90.388905,1.046851
6,Farmgate,23.757688,90.390176,0.325022
7,Kamal Sarani,23.786766,90.366914,4.192905
8,National Museum,23.779784,90.368032,3.491659
9,Sher-e-Bangla Road,23.775074,90.374060,2.706998


# *Graph visualization with heuristics and  path costs*

In [18]:
from pyvis.network import Network
import pandas as pd
import networkx as nx
from IPython.core.display import display, HTML
net=Network(notebook=True)
G = nx.Graph()
for location in location_data.values.tolist():
    loc_label=location[0]+' h(n) = '+str(round(heuristics[location[0]],3))
    if location[0]==source[0]:
        #net.add_node(location[0],label=str(location[0]),color='blue')
        G.add_node(location[0],color="green",label=loc_label)
    elif location[0]==goal[0]:
        #net.add_node(location[0],label=str(location[0]),color='green')
        G.add_node(location[0],color="red",label=loc_label)
    else:
        #net.add_node(location[0],label=str(location[0]))
        G.add_node(location[0],label=loc_label)

for location in location_graph.values.tolist():
    locationA,locationB=location[0],location[1]
    G.add_edge(locationA,locationB,weight=float(location[2])*10, title=str(location[2]), label=str(round(location[2],5))+'KM')
    #G[locationA][locationB]['weight']=str(location[2])
    #net.add_edge(locationA,locationB,value=str(location[2]),title=str(location[2])+' KM')
net.from_nx(G,show_edge_weights=True)
net.toggle_physics(False)
net.show(name='./graph.html',notebook=True)
# display(HTML('./graph.html'))   


./graph.html


C:\Users\mfttu\AppData\Local\Temp\ipykernel_23580\4132986588.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Green Indicates starting position which is in my case 'Organ Lily' and red indicates UAP.

# *The A * Search Algorithm*

The A* search algorithm is a widely used algorithm in artificial intelligence and computer science for finding the shortest path between two points on a graph. It is an extension of the Dijkstra's algorithm, which is a popular algorithm for finding the shortest path between two points in a graph with non-negative edge weights.

Here are the steps involved in the A* search algorithm:

- Initialize the start node and the goal node.
- Create an open set and a closed set. The open set contains nodes that need to be explored, while the closed set contains nodes that have already been explored.
- Add the start node to the open set.
- While the open set is not empty, do the following:
    - Find the node with the lowest f-score in the open set. This node will be the current node.
    - If the current node is the goal node, return the path to it.
    - Remove the current node from the open set and add it to the closed set.
    - For each neighbor of the current node, do the following:
        - If the neighbor is in the closed set, skip to the next neighbor.
        - If the neighbor is not in the open set, add it to the open set and set its parent to the current node. Compute its g-score and h-score.
        - If the neighbor is already in the open set, check if the new g-score is lower than the existing g-score. If it is, update the neighbor's parent and g-score.
- If the open set is empty and the goal node has not been reached, return failure.


The f-score of a node is the sum of its g-score (the cost of getting to that node from the start node) and its h-score (an estimate of the cost of getting from that node to the goal node). The h-score is typically computed using a heuristic function that estimates the distance between the current node and the goal node.

In [19]:
from collections import defaultdict
import heapq
score_graph=defaultdict(int)
def a_star_algorithm(edges:dict[str],
                     heuristic:dict[str],
                     source:str,
                     goal:str)->list[int]:
  
    '''
    A star search algorithm
    takes in edges,heuristic,source and goal 
    returns list and optimal cost
    if nothing is found then returns a empty list
    '''
    g_score=defaultdict(lambda:float('Inf'))
    f_score=defaultdict(lambda:float('Inf'))
    in_the_fringe=defaultdict(lambda:False)

    f_score[source]=heuristic[source]
    g_score[source]=0
    open_fringe=[(f_score[source],source)]
    path=defaultdict(lambda:False)
    close_fringe=[]

    path_found=False
    
    
    optimal_cost=0
    while len(open_fringe)>0:
        
        _,node=heapq.heappop(open_fringe)
        
        if node==goal:
            path_found=True
            break
        if node in edges:
            for adjacent in edges[node]:
            
                adjacent_node,cost_to_adjacent=adjacent
                if g_score[node]+cost_to_adjacent<g_score[adjacent_node]:
                    g_score[adjacent_node]=g_score[node]+cost_to_adjacent
                    score_graph[(adjacent_node,node)]=cost_to_adjacent
                    
                    f_score[adjacent_node]=g_score[adjacent_node]+heuristic[adjacent_node]
                    path[adjacent_node]=node
                    
                    adjacent_data=str(adjacent_node)+str(f_score[adjacent_node])
                    if adjacent_data not in in_the_fringe:
                        in_the_fringe[adjacent_data]=True
                        heapq.heappush(open_fringe,(f_score[adjacent_node],adjacent_node))
        
        #print(f"{node}: {open_fringe}")
    
    if path_found:
        total_path=[]
        source=goal
        total_path.append(goal)
        while path[source]:
            optimal_cost+=score_graph[(source,path[source])]
            source=path[source]
            total_path.append(source)
        total_path.reverse()
        return total_path,optimal_cost
    else:
        return []

***Let us see our graph as python dictionary***

In [20]:
graph

{'Organ Lily': [('Shewrapara', 1.1657375229274665),
  ('Kamal Sarani', 1.3529079354871336)],
 'Shewrapara': [('Agargaon', 1.2841500686079175)],
 'Agargaon': [('Chandrima Uddan', 1.50979314681242)],
 'Chandrima Uddan': [('Khamar Bari', 0.7054088572099919),
  ('Bijoy Sarani', 0.5930516249772503)],
 'Khamar Bari': [('Farmgate', 0.6518919580040842)],
 'Bijoy Sarani': [('Farmgate', 0.7542073013508552)],
 'Farmgate': [('University of Asia Pacific', 0.3250216409379184)],
 'Kamal Sarani': [('National Museum', 0.7847256846643799)],
 'National Museum': [('Sher-e-Bangla Road', 0.8065672061887167)],
 'Sher-e-Bangla Road': [('Chandrima Uddan', 1.4302531679306827)]}

***Heurisitics***

In [21]:
heuristics

{'Organ Lily': 5.211021625426437,
 'Shewrapara': 4.06288787306962,
 'Agargaon': 2.778856717736909,
 'Chandrima Uddan': 1.2937189139764316,
 'Khamar Bari': 0.6854378233183538,
 'Bijoy Sarani': 1.0468513809502136,
 'Farmgate': 0.3250216409379184,
 'Kamal Sarani': 4.192905233157257,
 'National Museum': 3.491658511492129,
 'Sher-e-Bangla Road': 2.7069978872491887,
 'University of Asia Pacific': 0.0}

***Goal Node***

In [22]:
goal

Location     University of Asia Pacific
Latitude                      23.754956
Longitude                     90.389039
Name: 10, dtype: object

***Starting position***

In [23]:
source

Location     Organ Lily
Latitude      23.798547
Longitude     90.370238
Name: 0, dtype: object

##### Passing  all the data to our a_star_algorithm_function to calculate the shortest path*

In [24]:
path,optimal_cost=a_star_algorithm(graph,heuristics,source[0],goal[0])
path_="->".join(path)


In [25]:
print(f"Path: {path_}")

Path: Organ Lily->Shewrapara->Agargaon->Chandrima Uddan->Bijoy Sarani->Farmgate->University of Asia Pacific


In [26]:
print(f"Optimal Cost is : {optimal_cost} km")


Optimal Cost is : 5.631961305613828 km


##### Visualizing the shortest path


In [27]:
from pyvis.network import Network
from IPython.core.display import display, HTML
net=Network(notebook=True)
for location in location_data.values.tolist():
    if location[0]==source[0]:
        net.add_node(location[0],label=location[0],color="green")
    
    if location[0]==goal[0]:
        net.add_node(location[0],label=location[0],color="magenta")
    else:
        net.add_node(location[0],label=location[0])
for i in range(1,len(path)):
    net.add_edge(path[i-1],path[i],color="red")
    
for location in location_graph.values.tolist():
    locationA,locationB=location[0],location[1]
    net.add_edge(locationA,locationB)
net.show('./final_graph.html')
net.show(name='./final_graph.html',notebook=True)
# display(HTML('./final_graph.html'))  

./final_graph.html
./final_graph.html


C:\Users\mfttu\AppData\Local\Temp\ipykernel_23580\737039.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


##### More visualization with path cost

In [28]:
from pyvis.network import Network
from IPython.core.display import display, HTML
net=Network(notebook=True)
G = nx.Graph()
for location in location_data.values.tolist():
    if location[0]==source[0]:
        G.add_node(location[0],label=location[0],color="green")
    
    if location[0]==goal[0]:
        G.add_node(location[0],label=location[0],color="magenta")
    else:
        G.add_node(location[0],label=location[0])
    #net.add_node(location[0],label=location[0])
for i in range(1,len(path)):
    G.add_edge(path[i-1],path[i],color='red',label=str(round(score_graph[(path[i],path[i-1])],5))+' KM')
    #net.add_edge(path[i-1],path[i],color="red")
    
for location in location_graph.values.tolist():
    locationA,locationB=location[0],location[1]
    G.add_edge(locationA,locationB)
    #net.add_edge(locationA,locationB)
net.from_nx(G,show_edge_weights=True)
net.show(name='./final_graph.html',notebook=True)
# display(HTML('./final_graph.html'))  

./final_graph.html


C:\Users\mfttu\AppData\Local\Temp\ipykernel_23580\3602060340.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [30]:
print(f"Optimal Cost is : {optimal_cost} km")


Optimal Cost is : 5.631961305613828 km
